In [1]:
import numpy as np
import matplotlib
from mido import Message, MetaMessage
import mido
import sys
import random
import os
import time
import math
pwd = "/home/zedroman/Documents/Sonia_Data/Songs/maestro/"
savepwd="/home/zedroman/Documents/Sonia_Data/Songs/"
music = sorted([f for f in os.listdir(pwd) if f[len(f)-4:len(f)] == ".mid"])
sos = 0
eos = 1
message_type = {"note_on":1, "note_off":0}
type_message = {1:"note_on", 0:"note_off"}
tempo = 500000 * 5
hidden_size = 750

In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
device="cpu"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
def create_random(music):
    randoms = []
    for i in range(len(music)):
        randoms.append([random.random() * 10 for i in range(hidden_size)])
    return randoms
random_starts = create_random(music)

In [4]:
# 0(sos) 1(eos) 2 - 129(note_on) 130 - 257(note_off) 258 - 385(velocity) 386-585 //386-395,396-405, 406-415, 416-425 (time_shift)

def encode_type(tp, num):
    if (tp == "sos"):
        return 0
    if (tp == "eos"):
        return eos
    if (tp == "note_on"):
        return num + 2
    if (tp == "note_off"):
        return num + 130
    if (tp == "velocity"):
        return num + 258
    if (tp == "time"):
        return (num // 8) + 386
    return -1

def get_type(c):
    if(c == 0):
        return "sos", c
    if(c == 1):
        return "eos", c
    if(c > 1 and c <= 129):
        return "note_on", c - 2
    
    if(c >= 130 and c < 258):
        return "note_off", c - 130
    
    if(c>=258 and c < 386):
        return "velocity", c - 258
    
    if(c>=386 and c < 586):
        return "time", int((c - 386) * 8)
    
    print(c)
    return "wtf", -1
    


def midi_to_list(mid=None, pwd=None, tensor=True):
    if(mid==None):
          mid = mido.MidiFile(pwd)
    sample=[]
    velocity = -1
    for i, track in enumerate(mid.tracks):
        j = 0;
        for msg in track:
            j += 1
            if (j > 3000):
                break
            if (msg.type == "note_off" or msg.type == "note_on"):
                if (msg.time != 0):
                    c= encode_type("time", msg.time)
                    if (c != -1):
                        sample.append(c)
#                         for i in c:
#                             sample.append(i)
                if (msg.velocity != velocity):
                    c = encode_type("velocity", msg.velocity)
                    if (c != -1): 
                        sample.append(c)
                    velocity = msg.velocity
                if (msg.type == "note_on"):
                    c = encode_type("note_on", msg.note)
                    if (c != -1): 
                        sample.append(c)
                if (msg.type == "note_off"):
                    c = encode_type("note_off", msg.note)
                    if (c != -1): 
                        sample.append(c)
    sample.append(eos)
    if (tensor):
        return torch.tensor(sample, device=device)
    else:
        return sample
#     return sorted(sample, key=lambda x: x[-1])

def check_midi(sample):
    note_on = {}
    dele = []
    for j,i in enumerate(sample):
#         print(i)
        tp, nt = get_type(i)
        if (tp == "note_on"):
#             if(nt in note_on):
#                 dele.append(j);
            note_on[nt] = 1;
        if (tp == "note_off"):
            if(nt in note_on):
                del note_on[nt]
            else:
                dele.append(j)
    for i in range(len(dele) - 1, -1, -1):
        del(sample[dele[i]])
    for i in note_on:
        sample.append(encode_type("note_off", i))

def list_to_midi(sample):
    print("was: ",len(sample))
    check_midi(sample)
    print("now: ",len(sample))
    
    midi = mido.MidiFile(type=0)
    tr = mido.MidiTrack()
    midi.tracks.append(tr)
    tr.append(MetaMessage("set_tempo", tempo=tempo, time=0))
    velocity = 0
    time = 0
    for i in sample:
        tp, nt = get_type(i)
        if (tp == "velocity"):
            velocity = nt
        if (tp == "time"):
            time += nt
        if (tp == "note_off" or tp == "note_on"):
#             print(nt, velocity, time)
            tr.append(Message(tp, note=nt, velocity=velocity, time=time, channel=0))
            time = 0
    return midi

In [5]:
for file in (music):
    midi_file = mido.MidiFile(pwd + file)
    sample = midi_to_list(mid=midi_file, tensor=False)
    midi = list_to_midi(sample)

was:  1609
now:  1641
was:  1659
now:  1694
was:  1644
now:  1691
was:  685
now:  714
was:  1377
now:  1417
was:  1621
now:  1678
was:  1643
now:  1711
was:  1597
now:  1659
was:  1959
now:  2013
was:  1446
now:  1487
was:  1737
now:  1807
was:  1426
now:  1462
was:  1566
now:  1608
was:  1334
now:  1377
was:  1560
now:  1593
was:  1970
now:  2027
was:  2045
now:  2094
was:  1736
now:  1787
was:  1682
now:  1746
was:  1624
now:  1682
was:  2047
now:  2105
was:  2039
now:  2084
was:  2028
now:  2078
was:  1642
711
now:  1704
711
was:  1033
now:  1065
was:  2096
now:  2137
was:  1429
now:  1465
was:  1128
601
now:  1171
601
was:  714
now:  749
was:  1549
now:  1587
was:  749
now:  784
was:  1564
now:  1594
was:  1585
now:  1624
was:  1116
now:  1162
was:  836
now:  876
was:  10
now:  14
was:  2007
now:  2050
was:  1852
now:  1896
was:  1753
now:  1793
was:  2038
now:  2097
was:  1423
now:  1461
was:  1363
now:  1394
was:  1773
now:  1812
was:  1999
now:  2040
was:  1859
now:  1920
was:  

now:  95
was:  2341
now:  2374
was:  2295
now:  2333
was:  2242
now:  2286
was:  1959
now:  2014
was:  2457
now:  2493
was:  1402
now:  1421
was:  2673
now:  2710
was:  2816
now:  2860
was:  2695
now:  2735
was:  2446
798
now:  2489
798
was:  2167
now:  2211
was:  2184
now:  2224
was:  1095
now:  1128
was:  1121
now:  1153
was:  1765
now:  1808
was:  2147
now:  2180
was:  1526
now:  1564
was:  1423
now:  1467
was:  1437
now:  1479
was:  1198
now:  1248
was:  1179
now:  1223
was:  1558
now:  1597
was:  2346
now:  2390
was:  2240
now:  2290
was:  1999
now:  2039
was:  1817
now:  1876
was:  1118
now:  1159
was:  1302
now:  1326
was:  2398
now:  2436
was:  1028
now:  1066
was:  2087
now:  2136
was:  1803
now:  1839
was:  1331
now:  1382
was:  1115
now:  1156
was:  147
now:  163
was:  2461
now:  2506
was:  1971
now:  2025
was:  2155
now:  2211
was:  2232
now:  2284
was:  813
now:  863
was:  1893
now:  1919
was:  2105
now:  2150
was:  1616
now:  1657
was:  1819
now:  1873
was:  2038
now:  20

was:  2246
now:  2317
was:  2317
now:  2379
was:  923
now:  974
was:  763
now:  782
was:  666
now:  687
was:  714
now:  738
was:  678
now:  698
was:  1305
now:  1345
was:  690
now:  717
was:  1583
899
now:  1642
899
was:  2103
now:  2166
was:  1158
now:  1202
was:  564
616
now:  584
616
was:  639
now:  667
was:  1229
now:  1288
was:  647
now:  683
was:  1625
now:  1686
was:  2175
now:  2238
was:  1294
now:  1328
was:  2076
now:  2107
was:  2072
now:  2124
was:  1590
now:  1655
was:  1803
now:  1867
was:  216
751
now:  236
751
was:  2111
now:  2154
was:  1869
now:  1911
was:  1914
now:  1964
was:  2160
now:  2211
was:  1591
now:  1636
was:  1846
now:  1893
was:  356
788
746
now:  383
788
746
was:  1702
now:  1748
was:  1491
now:  1516
was:  1786
now:  1828
was:  1848
now:  1897
was:  1932
now:  1998
was:  1806
now:  1866
was:  1789
now:  1839
was:  1625
now:  1667
was:  2133
now:  2193
was:  1712
now:  1758
was:  2431
now:  2478
was:  1965
now:  2021
was:  1806
now:  1870
was:  1666
now

was:  1269
now:  1307
was:  1103
now:  1139
was:  1023
now:  1060
was:  1495
now:  1531
was:  362
now:  389
was:  1826
now:  1870
was:  1325
now:  1355
was:  2015
now:  2056
was:  1979
now:  2033
was:  1933
now:  1994
was:  1925
now:  1966
was:  2073
now:  2126
was:  1756
now:  1788
was:  2246
now:  2288
was:  2113
645
now:  2154
645
was:  1663
now:  1708
was:  1900
now:  1945
was:  1748
now:  1797
was:  1993
now:  2051
was:  1997
now:  2051
was:  2017
now:  2054
was:  1920
now:  1959
was:  1511
now:  1548
was:  1689
now:  1735
was:  2009
622
now:  2049
622
was:  1674
now:  1716
was:  2088
now:  2135
was:  2051
now:  2102
was:  1258
now:  1299
was:  1673
now:  1723
was:  1715
now:  1760
was:  1655
now:  1694
was:  1483
now:  1535
was:  2017
now:  2069
was:  1677
now:  1716
was:  1279
now:  1321
was:  788
now:  812
was:  1869
now:  1908
was:  2202
now:  2256
was:  1883
now:  1930
was:  2188
now:  2238
was:  2149
now:  2202
was:  1763
now:  1807
was:  1365
now:  1417
was:  1723
now:  177

was:  1689
now:  1731
was:  730
864
767
now:  757
864
767
was:  541
now:  564
was:  534
now:  561
was:  562
now:  587
was:  448
now:  472
was:  489
now:  521
was:  693
now:  721
was:  738
now:  769
was:  689
now:  716
was:  721
now:  750
was:  1909
now:  1956
was:  729
now:  759
was:  751
now:  779
was:  634
886
now:  660
886
was:  1
now:  1
was:  1928
now:  1967
was:  1791
now:  1840
was:  2123
now:  2183
was:  2110
now:  2177
was:  2178
now:  2246
was:  1972
now:  2030
was:  1947
now:  1990
was:  2180
now:  2235
was:  2167
now:  2232
was:  2161
now:  2232
was:  2066
now:  2138
was:  1257
596
now:  1313
596
was:  1817
635
now:  1875
635
was:  1830
now:  1867
was:  2168
684
now:  2219
684
was:  1653
now:  1712
was:  2025
now:  2061
was:  1978
now:  2023
was:  1152
now:  1201
was:  839
now:  876
was:  1561
now:  1599
was:  1314
now:  1364
was:  921
now:  959
was:  1721
now:  1761
was:  2177
592
now:  2245
592
was:  2224
now:  2288
was:  1810
now:  1852
was:  1414
664
now:  1468
664
was:

now:  1412
was:  1683
now:  1746
was:  1360
now:  1417
was:  1745
now:  1805
was:  1886
now:  1928
was:  1798
now:  1841
was:  1998
now:  2056
was:  1477
now:  1520
was:  481
638
now:  523
638
was:  796
now:  828
was:  1153
now:  1192
was:  1277
now:  1314
was:  1592
now:  1634
was:  2064
now:  2115
was:  1563
now:  1603
was:  1594
now:  1637
was:  1929
now:  1984
was:  2223
now:  2276
was:  1543
now:  1593
was:  2067
now:  2107
was:  1544
now:  1584
was:  928
now:  963
was:  1004
now:  1037
was:  2294
now:  2334
was:  2011
now:  2056
was:  1504
now:  1552
was:  837
now:  879
was:  1043
now:  1085
was:  2039
now:  2088
was:  1490
now:  1517
was:  2035
now:  2090
was:  2393
now:  2448
was:  2234
now:  2267
was:  1893
now:  1928
was:  2314
now:  2360
was:  510
784
now:  545
784
was:  2140
now:  2191
was:  2146
now:  2199
was:  1926
now:  1976
was:  1750
now:  1784
was:  2428
now:  2483
was:  2059
now:  2127
was:  2091
now:  2149
was:  1567
now:  1608
was:  1263
now:  1309
was:  1679
now:

In [6]:

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [7]:
max_len = 1000
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
#         self.softmax = nn.Softmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self, num):
        if (num != -1):
            random.seed(num)
        rand = random.random() * 10
#         if (num == -1):
        print(rand)
        return  torch.tensor([rand]*decoder.hidden_size, device=device).view(1,1,-1)

In [8]:
teacher_forcing_ratio = 1


def train(target_tensor, decoder, decoder_optimizer, criterion, max_length=max_len, num = -1):
    decoder_optimizer.zero_grad()
    target_length = target_tensor.size(0)

    loss = 0

    decoder_input = torch.tensor([[sos]], device=device)
    decoder_hidden =   torch.tensor(random_starts[num], device=device).view(1,1,-1)

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
#             if di == max_len:
#                 break
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di:di+1])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di:di+1])
            if decoder_input.item() == eos:
                break

    loss.backward()

    decoder_optimizer.step()

    return loss.item() / target_length

In [9]:
def trainIters(decoder, n_iters,decoder_optimizer, print_every=len(music), epoch=0):
    start = time.time()
    print_loss_total = 0  # Reset every print_every

    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = music[iter % (len(music))]
        print(training_pair)
        midi_file = mido.MidiFile(pwd + training_pair)
        target_tensor = midi_to_list(mid=midi_file)
        del midi_file
        loss = train(target_tensor,
                     decoder, decoder_optimizer, criterion, num = iter % (len(music)))
        print_loss_total += loss
        del target_tensor
        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

In [10]:
learning_rate = 0.001
decoder = DecoderRNN(hidden_size, 586).to(device)
decoder_optimizer = optim.Adam(decoder.parameters() , lr=learning_rate)


In [12]:
trainIters(decoder, 10000, decoder_optimizer, print_every=5)

1.mid


RuntimeError: CUDA error: device-side assert triggered

In [ ]:
def evaluate(decoder,num=-1, max_length=max_len):
    with torch.no_grad():

        decoder_input = torch.tensor([[sos]], device=device)  # SOS
        decoder_hidden = [random.random() for i in range(hidden_size)]
        decoder_hidden =   torch.tensor(decoder_hidden, device=device).view(1,1,-1)

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            topv = topv.max().item() - topv.cpu()[0].numpy()
            summ = topv.sum()
            chosen = np.random.choice(topi[0].cpu(), p= topv / summ)
            decoder_input = torch.tensor(chosen, device=device)
            if decoder_input == eos:
#                 decoded_words.append('<EOS>')
#                 break
                decoder_input = torch.tensor([[sos]], device=device)
            else:
                decoded_words.append(chosen)

#             decoder_input = topi.data.topk(1)[1].squeeze().detach()

        return decoded_words

In [ ]:
def generate(number, decoder):
    for i in range(number):
        new_music = evaluate(decoder)
        midi = list_to_midi(new_music)
        print(midi.length)
        midi.save(savepwd +"test" + str(i) + ".mid")
    return midi

In [ ]:

last = generate(10, decoder)
for track in last.tracks:
    for msg in track:
        print(msg)

In [ ]:

for i in new_music:
    print(get_type(i))

In [ ]:
def test():
    midi = mido.MidiFile("/home/zedroman/Documents/Sonia_Data/Songs/test_from.mid")
#     midi = mido.MidiFile()
#     tr = mido.MidiTrack()
#     midi.tracks.append(tr)
#     tr.append(Message("note_on", velocity = 10, time= 1111, channel=0))
    sample = midi_to_list(midi, tensor=False)
#     for i in sample:
#         print(i)
    midi = list_to_midi(sample)
    midi.save("/home/zedroman/Documents/Sonia_Data/Songs/test_to.mid")
tempo = 500000 * 5
test()

In [ ]:
midi = mido.MidiFile(pwd + "2.mid")
for i, track in enumerate(midi.tracks):
    print('Track {}: {}'.format(i, track.name))
    for msg in track:
        print(msg)
# tmp = music[0]
# music[0] =music[2]
# music[2] = tmp

In [ ]:
print(music)

In [ ]:
torch.save(decoder, '/home/zedroman/Documents/Sonia_Data/test_models4/decoder')

In [ ]:
decoder = torch.load('/home/zedroman/Documents/Sonia_Data/test_models4/decoder')